In [88]:
#takes 
from __future__ import division, absolute_import, print_function
%load_ext autoreload
%autoreload 2
import os
os.chdir(r"D:\data\ortoCC\design-with-alignments\4heptade-7")
print(os.getcwd())
import hepran
import hepran.bzipscore as bz
from hepran.bzipscore import *
import hepran.utils as u
import hepran.registers as r
import numpy as np
CCO = os.environ.get("CCO",r"C:\Projekti_KI\ortoCC\CoiledCoilOrtho")
from glob import glob
import pandas as pd
from StringIO import StringIO
import collections

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
D:\data\ortoCC\design-with-alignments\4heptade-7


In [83]:
sets = glob("!OUT_bcf/*.set")  
sets += glob("!OUT_additional/*.set")  

final_fasta = collections.OrderedDict()
final_pairs = []

In [84]:
for _set in sets:
    set_data = u.load_set(_set)
    fasta_data = u.load_fasta(u.replace_extension(_set,'fasta'))
    
    #change first IDs to D
    fasta_data = u.change_nth_res(fasta_data, pos=0, res='D')
    #ids are loaded from set. Important, because fasta can then be larger...
    set_ids = bz.get_ids_from_pairs(set_data)
    all_pairs = u.make_all_pairs(set_ids, is_symmetric=True)
    u.append_pairs(final_pairs, all_pairs, is_symmetric=True)
    u.append_fasta(final_fasta, fasta_data)
    

print("pairs:",len(final_pairs))
print("IDs:"  ,len(final_fasta))

u.save_dict_to_fasta(final_fasta, "!all-sym.fasta")
u.save_set(final_pairs, "!all-sym.pairs")

pairs: 8155
IDs: 736


In [85]:
## Load the pairs for which we have experimental data

seqs_df = pd.read_excel("!OUT_additional_pairs/reliable-seq+Tm.xlsx", sheetname="sequence")
#throw out whitespace and -
#warning - alignment is lost!
seqs_df.drop(0, inplace=True)
seqs_df.sequence = seqs_df.sequence.str.replace(" ", "")
seqs_df.sequence = seqs_df.sequence.str.replace("-", "")
#Limit to 
seqs_df.sequence = seqs_df.sequence.str[0:29]
seqs_exp = collections.OrderedDict()
for i, row in seqs_df.iterrows():
    seqs_exp[row.peptide] = row.sequence
    
seqs_exp;

pairs_df = pd.read_excel("!OUT_additional_pairs/reliable-seq+Tm.xlsx", sheetname="Tm")
pairs_df
pairs_exp = [(row.peptide1, row.peptide2) for i, row in pairs_df.iterrows()]
pairs_exp;

##append the pairs
u.append_pairs(final_pairs, pairs_exp, is_symmetric=True)
u.append_fasta(final_fasta, seqs_exp)
    

print("pairs:",len(final_pairs))
print("IDs:"  ,len(final_fasta))

u.save_dict_to_fasta(final_fasta, "!all-sym.fasta")
u.save_set(final_pairs, "!all-sym.pairs")

pairs: 8169
IDs: 761


In [91]:
assert u.check_pairs_and_fasta(final_fasta, final_pairs)